2Fish- logistic_map

In [ ]:
import time
import secrets
from base64 import b64encode, b64decode
from hashlib import sha256
from twofish import Twofish

# === Baudot Encoding (a-z and space only) ===
baudot = {
    ' ': '00000', 't': '00001', 'o': '00011', 'h': '00101', 'n': '00110',
    'm': '00111', 'l': '01001', 'r': '01010', 'g': '01011', 'i': '01100',
    'p': '01101', 'c': '01110', 'v': '01111', 'e': '10000', 'z': '10001',
    'd': '10010', 'b': '10011', 's': '10100', 'y': '10101', 'f': '10110',
    'x': '10111', 'a': '11000', 'w': '11001', 'j': '11010', 'u': '11100',
    'q': '11101', 'k': '11110'
}
reverse_baudot = {v: k for k, v in baudot.items()}

def to_baudot_bits(text):
    return ''.join([baudot[c] for c in text.lower() if c in baudot])

def bits_to_bytes(bits):
    while len(bits) % 8 != 0:
        bits += '0'
    return [int(bits[i:i+8], 2) for i in range(0, len(bits), 8)]

def bytes_to_bits(byte_list):
    return ''.join([format(b, '08b') for b in byte_list])

def from_baudot_bits(bits):
    return ''.join([reverse_baudot.get(bits[i:i+5], '?') for i in range(0, len(bits), 5)])

# === Logistic Map for chaos ===
def logistic_map(x0, delta, length):
    x = x0
    chaos = []
    for _ in range(length):
        x = delta * x * (1 - x)
        chaos.append(int(x * 256) % 256)
    return chaos

def xor_lists(a, b):
    return [x ^ y for x, y in zip(a, b)]

# === Twofish Encryption ===
def twofish_encrypt(byte_list, key):
    key = sha256(key).digest()[:16]  # 128-bit key
    tf = Twofish(key)
    pad_len = (16 - len(byte_list) % 16) % 16
    byte_list += [pad_len] * pad_len
    ciphertext = b''
    for i in range(0, len(byte_list), 16):
        block = bytes(byte_list[i:i+16])
        ciphertext += tf.encrypt(block)
    return b64encode(ciphertext).decode()

def twofish_decrypt(data_b64, key):
    key = sha256(key).digest()[:16]
    tf = Twofish(key)
    raw = b64decode(data_b64)
    decrypted = []
    for i in range(0, len(raw), 16):
        decrypted += list(tf.decrypt(raw[i:i+16]))
    pad_len = decrypted[-1]
    if all(p == pad_len for p in decrypted[-pad_len:]):
        decrypted = decrypted[:-pad_len]
    return decrypted

# === Encryption ===
def encrypt(text, key, x0=0.7, delta=3.9):
    print(f"\n🔐 ENCRYPTING TEXT: {text[:50]}...")
    bits = to_baudot_bits(text)
    print(f"Baudot Bits: {bits[:80]}...")
    baudot_bytes = bits_to_bytes(bits)
    print(f"Baudot Bytes: {baudot_bytes[:10]}...")
    chaos = logistic_map(x0, delta, len(baudot_bytes))
    print(f"Logistic Chaotic Bytes: {chaos[:10]}...")
    xored = xor_lists(baudot_bytes, chaos)
    print(f"XORed Bytes: {xored[:10]}...")
    encrypted_b64 = twofish_encrypt(xored, key)
    print(f"🔒 Twofish Encrypted (Base64): {encrypted_b64[:80]}...")
    print(f"🧮 Baudot Bit Length: {len(bits)} (save this for decryption)")
    return encrypted_b64, len(bits)

# === Decryption ===
def decrypt(enc_b64, key, bit_len, x0=0.7, delta=3.9):
    print("\n🔓 DECRYPTING...")
    xored = twofish_decrypt(enc_b64, key)
    print(f"Decrypted XORed Bytes: {xored[:10]}...")
    chaos = logistic_map(x0, delta, len(xored))
    print(f"Regenerated Chaotic Bytes: {chaos[:10]}...")
    baudot_bytes = xor_lists(xored, chaos)
    print(f"Recovered Baudot Bytes: {baudot_bytes[:10]}...")
    bits = bytes_to_bits(baudot_bytes)[:bit_len]
    print(f"Baudot Bits: {bits[:80]}...")
    text = from_baudot_bits(bits).upper()
    print(f"✅ Decrypted Text Preview: {text[:80]}...")
    return text

def generate_random_key(length=16):
    return secrets.token_bytes(length)

# === Main ===
def main():
    file_path = "/content/200.txt"  # ⚠️ Update if needed
    with open(file_path, 'r') as f:
        raw_text = f.read().lower()

    # Clean input
    text = ''.join(c for c in raw_text if c in baudot)

    tf_key = generate_random_key()
    print(f"\n🗝 Random Twofish Key Used (Hex): {tf_key.hex()}")

    # Encryption
    start_enc = time.time()
    enc_text, bit_len = encrypt(text, tf_key)
    end_enc = time.time()

    # Decryption
    start_dec = time.time()
    decrypted = decrypt(enc_text, tf_key, bit_len)
    end_dec = time.time()

    print(f"\n⏱ Encryption Time: {(end_enc - start_enc) * 1000:.2f} ms")
    print(f"⏱ Decryption Time: {(end_dec - start_dec) * 1000:.2f} ms")

if __name__ == "__main__":
    main()


2Fish- Tent Map

In [ ]:
import time
import secrets
from base64 import b64encode, b64decode
from hashlib import sha256
from Crypto.Util.Padding import pad, unpad
from twofish import Twofish

# Baudot encoding for a-z and space only
baudot = {
    ' ': '00000', 't': '00001', 'o': '00011', 'h': '00101', 'n': '00110',
    'm': '00111', 'l': '01001', 'r': '01010', 'g': '01011', 'i': '01100',
    'p': '01101', 'c': '01110', 'v': '01111', 'e': '10000', 'z': '10001',
    'd': '10010', 'b': '10011', 's': '10100', 'y': '10101', 'f': '10110',
    'x': '10111', 'a': '11000', 'w': '11001', 'j': '11010', 'u': '11100',
    'q': '11101', 'k': '11110'
}
reverse_baudot = {v: k for k, v in baudot.items()}

def to_baudot_bits(text):
    return ''.join([baudot[c] for c in text.lower() if c in baudot])

def bits_to_bytes(bits):
    while len(bits) % 8 != 0:
        bits += '0'
    return [int(bits[i:i+8], 2) for i in range(0, len(bits), 8)]

def bytes_to_bits(byte_list):
    return ''.join([format(b, '08b') for b in byte_list])

def from_baudot_bits(bits):
    return ''.join([reverse_baudot.get(bits[i:i+5], '?') for i in range(0, len(bits), 5)])

# Tent map for chaos generation
def tent_map(x0, r, length):
    x = x0
    chaos = []
    for _ in range(length):
        x = r * x if x < 0.5 else r * (1 - x)
        chaos.append(int(x * 256) % 256)
    return chaos

def xor_lists(a, b):
    return [x ^ y for x, y in zip(a, b)]

# === Twofish encryption ===
def twofish_encrypt(byte_list, key):
    key = sha256(key).digest()[:16]  # 128-bit key
    tf = Twofish(key)
    pad_len = (16 - len(byte_list) % 16) % 16
    byte_list += [pad_len] * pad_len

    encrypted = b''
    for i in range(0, len(byte_list), 16):
        block = bytes(byte_list[i:i+16])
        encrypted += tf.encrypt(block)

    return b64encode(encrypted).decode()

def twofish_decrypt(data_b64, key):
    key = sha256(key).digest()[:16]
    tf = Twofish(key)
    raw = b64decode(data_b64)

    decrypted = []
    for i in range(0, len(raw), 16):
        block = raw[i:i+16]
        decrypted.extend(tf.decrypt(block))

    pad_len = decrypted[-1]
    if all(p == pad_len for p in decrypted[-pad_len:]):
        decrypted = decrypted[:-pad_len]

    return decrypted
# ===========================

def encrypt(text, key, x0=0.7, r=1.99):
    print(f"\n🔐 ENCRYPTING TEXT: {text[:50]}...")
    bits = to_baudot_bits(text)
    print(f"Baudot Bits: {bits[:80]}...")
    baudot_bytes = bits_to_bytes(bits)
    print(f"Baudot Bytes: {baudot_bytes[:10]}...")
    chaos = tent_map(x0, r, len(baudot_bytes))
    print(f"Tent Chaotic Bytes: {chaos[:10]}...")
    xored = xor_lists(baudot_bytes, chaos)
    print(f"XORed Bytes: {xored[:10]}...")
    enc = twofish_encrypt(xored, key)
    print(f"🔒 Twofish Encrypted (Base64): {enc[:80]}...")
    print(f"🧮 Baudot Bit Length: {len(bits)} (save this for decryption)")
    return enc, len(bits)

def decrypt(enc_b64, key, bit_len, x0=0.7, r=1.99):
    print("\n🔓 DECRYPTING...")
    xored = twofish_decrypt(enc_b64, key)
    print(f"Decrypted XORed Bytes: {xored[:10]}...")
    chaos = tent_map(x0, r, len(xored))
    print(f"Regenerated Tent Bytes: {chaos[:10]}...")
    baudot_bytes = xor_lists(xored, chaos)
    print(f"Recovered Baudot Bytes: {baudot_bytes[:10]}...")
    bits = bytes_to_bits(baudot_bytes)[:bit_len]
    print(f"Baudot Bits: {bits[:80]}...")
    text = from_baudot_bits(bits).upper()
    print(f"✅ Decrypted Text Preview: {text[:80]}...")
    return text

def generate_random_key(length=16):
    return secrets.token_bytes(length)

def main():
    file_path = "/content/uu.txt"  # Update this if needed
    with open(file_path, 'r') as f:
        raw_text = f.read().lower()

    # Filter to only valid Baudot chars
    text = ''.join(c for c in raw_text if c in baudot)

    tf_key = generate_random_key()
    print(f"\n🗝 Random Twofish Key Used (Hex): {tf_key.hex()}")

    # ENCRYPTION
    start_enc = time.time()
    enc_data, bit_len = encrypt(text, tf_key)
    end_enc = time.time()

    # DECRYPTION
    start_dec = time.time()
    decrypted_text = decrypt(enc_data, tf_key, bit_len)
    end_dec = time.time()

    print(f"\n⏱ Encryption Time: {(end_enc - start_enc) * 1000:.2f} ms")
    print(f"⏱ Decryption Time: {(end_dec - start_dec) * 1000:.2f} ms")

if __name__ == "__main__":
    main()


2Fish- Cosine Polynomial

In [ ]:
import time
import secrets
import math
from base64 import b64encode, b64decode
from hashlib import sha256
from twofish import Twofish

# Baudot encoding (a-z + space)
baudot = {
    ' ': '00000', 't': '00001', 'o': '00011', 'h': '00101', 'n': '00110',
    'm': '00111', 'l': '01001', 'r': '01010', 'g': '01011', 'i': '01100',
    'p': '01101', 'c': '01110', 'v': '01111', 'e': '10000', 'z': '10001',
    'd': '10010', 'b': '10011', 's': '10100', 'y': '10101', 'f': '10110',
    'x': '10111', 'a': '11000', 'w': '11001', 'j': '11010', 'u': '11100',
    'q': '11101', 'k': '11110'
}
reverse_baudot = {v: k for k, v in baudot.items()}

def to_baudot_bits(text):
    return ''.join([baudot[c] for c in text.lower() if c in baudot])

def bits_to_bytes(bits):
    while len(bits) % 8 != 0:
        bits += '0'
    return [int(bits[i:i+8], 2) for i in range(0, len(bits), 8)]

def bytes_to_bits(byte_list):
    return ''.join(format(b, '08b') for b in byte_list)

def from_baudot_bits(bits):
    return ''.join([reverse_baudot.get(bits[i:i+5], '?') for i in range(0, len(bits), 5)])

# ✅ Cosine Polynomial Map
def cosine_polynomial_map(x0, delta, length):
    x = x0
    chaos = []
    for _ in range(length):
        x = math.cos(delta * x**3 + x)
        x_norm = (x + 1) / 2  # Normalize from [-1, 1] to [0, 1]
        chaos.append(int(x_norm * 256) % 256)
    return chaos

def xor_lists(a, b):
    return [x ^ y for x, y in zip(a, b)]

# ✅ Twofish Encryption/Decryption
def twofish_encrypt(byte_list, key):
    key = sha256(key).digest()[:16]
    tf = Twofish(key)
    pad_len = (16 - len(byte_list) % 16) % 16
    byte_list += [pad_len] * pad_len
    ciphertext = b''
    for i in range(0, len(byte_list), 16):
        block = bytes(byte_list[i:i+16])
        ciphertext += tf.encrypt(block)
    return b64encode(ciphertext).decode()

def twofish_decrypt(data_b64, key):
    key = sha256(key).digest()[:16]
    tf = Twofish(key)
    raw = b64decode(data_b64)
    decrypted = []
    for i in range(0, len(raw), 16):
        block = raw[i:i+16]
        decrypted.extend(tf.decrypt(block))
    pad_len = decrypted[-1]
    if all(p == pad_len for p in decrypted[-pad_len:]):
        decrypted = decrypted[:-pad_len]
    return decrypted

def encrypt(text, key, x0=0.7, delta=1.2):
    print(f"\n🔐 ENCRYPTING TEXT: {text[:50]}...")
    bits = to_baudot_bits(text)
    print(f"Baudot Bits: {bits[:80]}...")
    baudot_bytes = bits_to_bytes(bits)
    print(f"Baudot Bytes: {baudot_bytes[:10]}...")
    chaos = cosine_polynomial_map(x0, delta, len(baudot_bytes))
    print(f"Cosine Chaotic Bytes: {chaos[:10]}...")
    xored = xor_lists(baudot_bytes, chaos)
    print(f"XORed Bytes: {xored[:10]}...")
    enc = twofish_encrypt(xored, key)
    print(f"🔒 Twofish Encrypted (Base64): {enc[:80]}...")
    print(f"🧮 Baudot Bit Length: {len(bits)} (save this for decryption)")
    return enc, len(bits)

def decrypt(enc_b64, key, bit_len, x0=0.7, delta=1.2):
    print("\n🔓 DECRYPTING...")
    xored = twofish_decrypt(enc_b64, key)
    print(f"Decrypted XORed Bytes: {xored[:10]}...")
    chaos = cosine_polynomial_map(x0, delta, len(xored))
    print(f"Regenerated Cosine Bytes: {chaos[:10]}...")
    baudot_bytes = xor_lists(xored, chaos)
    print(f"Recovered Baudot Bytes: {baudot_bytes[:10]}...")
    bits = bytes_to_bits(baudot_bytes)[:bit_len]
    print(f"Baudot Bits: {bits[:80]}...")
    text = from_baudot_bits(bits).upper()
    print(f"✅ Decrypted Text Preview: {text[:80]}...")
    return text

def generate_random_key(length=16):
    return secrets.token_bytes(length)

def main():
    file_path = "/content/uu.txt"  # Change this to your actual file path
    with open(file_path, 'r') as f:
        raw_text = f.read().lower()

    # Filter to valid Baudot chars
    text = ''.join(c for c in raw_text if c in baudot)

    tf_key = generate_random_key()
    print(f"\n🗝 Random Twofish Key Used (Hex): {tf_key.hex()}")

    # ENCRYPTION
    start_enc = time.time()
    encrypted, bit_len = encrypt(text, tf_key)
    end_enc = time.time()

    # DECRYPTION
    start_dec = time.time()
    decrypted_text = decrypt(encrypted, tf_key, bit_len)
    end_dec = time.time()

    print(f"\n⏱ Encryption Time: {(end_enc - start_enc) * 1000:.2f} ms")
    print(f"⏱ Decryption Time: {(end_dec - start_dec) * 1000:.2f} ms")

if __name__ == "__main__":
    main()
